# Transfer Learning

En el siguiente ejemplo, utilizaremos redes neuronales convolucionales para categorizar imágenes en distintos conjuntos de datos, aplicando técnicas de _transfer learning_. Dado que es altamente complejo y lento entrenar una red convolucional en conjunto de datos grandes como ImageNet, es muy común transferir lo aprendido en ese conjunto a otros sets de datos afines. En este ejemplo en particular, exploraremos dos casos de transferencia para realizar clasificación de imágenes que contienen abejas u hormigas:
- **Finetuning**: En vez de utilizar una inicialización aleatoria para los pesos de una red, inicializaremos los pesos (exceptuando los de la capa de clasificación) con los obtenidos para una ResNet18, después de haber sido entrenada en ImageNet. Luego de esto, el entrenamiento sigue como siempre.
- **Feature extraction**: Similar al caso anterior, con la diferencia que el entrenamiento solo considera la optimización de los pesos de la nueva capa de clasificación, es decir, todo el resto de los pesos queda fijo e igual a los obtenidos por la ResNet18 entrenada en ImageNet.

**IMPORTANTE**: al igual que en el ejemplo anterior, es importante que antes de ejecutar el código, estemos utilizando un _Runtime_ de tipo GPU. Para esto, deben seleccionar en el menú de arriba `Runtime -> Change Runtime Type -> GPU` y luego `Save`.



## 1. Importación de librerías

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import tqdm

import os
import copy

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#estas instrucciones sirven para aumentar el tamaño de gráficos e imágenes
plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams.update({'font.size': 16})

## 2. Carga, lectura y preprocesamiento de datos

En este ejemplo utilizaremos el set de datos _hymenoptera_, que contiene imágenes de abejas y hormigas, que deben ser clasificadas. El conjunto considera para cada categoría, 120 imágenes de entrenamiento y 75 de validación. Dado el reducido tamaño de este conjunto, es imposible enternar una CNN desde cero con estos datos, sin sufrir de serios problemas de sobreentrenamiento. Esto lo hace un candidato ideal para aplicar conceptos de _transfer learning_.

El primer paso consiste en descargar el set de datos y descomprimirlo, para lo que utilizamos los siguientes comandos:

In [ ]:
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip hymenoptera_data.zip
# es importante ejecutar esta celda solo una vez, de lo contrario se descargarán
# los datos multiples veces

A continuación, definimos los `Dataset` y `DataLoader` respectivos (entrenamiento y validación), considerando una serie de transformaciones de las imágenes, con el fin de hacer _data augmentation_ y reducir el sobreentrenamiento. Un aspecto importante a destacar es que los coeficientes utilizados para redimensionar y normalizar las imágenes son derivados del entrenamiento de una ResNet18 en ImageNet.

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

A continuación visualizamos algunas de las imágenes, con el fin de notar algunas de las transformaciones resultantes del proceso de _data augmentation_.

In [ ]:
def imshow(input_data, title=None):
    input_data = input_data.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input_data = std * input_data + mean
    input_data = np.clip(input_data, 0, 1)
    plt.imshow(input_data)
    plt.title(title)
    #plt.pause(0.001)  # pause a bit so that plots are updated

inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

## 3. Función de entrenamiento de los modelos

Dado que evaluaremos dos esquemas de entrenamiento distintos, es conveniente encapsular el process, con el fin evitar la repetición de código.

La función es similar a lo visto en los entrenamiento en ejemplos pasados, con la diferencia que acá se evalua el set de validación en cada época y se respalda el modelo que mejor rendimiento haya obtenido en el set de validación.

In [ ]:
def train_model(model, criterion, optimizer, device, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0
            total_examples = 0

            with tqdm.notebook.tqdm(total=len(dataloaders[phase]), unit='batch', 
                                    desc=f'Epoch {epoch+1}/{num_epochs} stage {phase}', 
                                    position=100, leave=True) as pbar: 
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  optimizer.zero_grad()

                  with torch.set_grad_enabled(phase == 'train'):
                      outputs = model(inputs)
                      _, preds = torch.max(outputs, 1)
                      loss = criterion(outputs, labels)

                      if phase == 'train':
                          loss.backward()
                          optimizer.step()

                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)
                  total_examples += inputs.size(0)

                  pbar.set_postfix(loss=running_loss/total_examples, 
                                   accuracy=running_corrects.item()/total_examples)
                  pbar.update()

              epoch_loss = running_loss / dataset_sizes[phase]
              epoch_acc = running_corrects.double() / dataset_sizes[phase]
              
              # deep copy the model
              if phase == 'val' and epoch_acc > best_acc:
                  best_acc = epoch_acc
                  best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

## 4. Finetuning

Para realizar el proceso de _finetuning_, primero descargamos la ResNet18 preentrenada. Luego, intercambiamos su capa de clasificación por una nueva, con el fin de clasificar entre las 2 categorías disponibles. Finalmente, definimos el resto de los elementos necesarios para el entrenamiento (pérdida y optimizador) y luego llamamos a la función de entrenamiento.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = models.resnet18(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# notar que acá se le entregan todos los parámetros del modelo al optimizador
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)

model_ft = train_model(model_ft, criterion, optimizer_ft, device, num_epochs=25)

## 5. Feature Extraction

Finalmente, evaluamos el segundo esquema de transferencia, al fijar los pesos de la red, mediante el uso de `requires_grad == False` en las capas de la ResNet18 y al pasar al optimizador solo los pesos de la capa lineal de clasificación.

In [ ]:
model_fe = torchvision.models.resnet18(pretrained=True)
for param in model_fe.parameters():
    param.requires_grad = False

num_ftrs = model_fe.fc.in_features
model_fe.fc = nn.Linear(num_ftrs, 2)

model_fe = model_fe.to(device)

criterion = nn.CrossEntropyLoss()

# notar que acá solo se pasan los pesos de la última capa densa
optimizer_fe = optim.Adam(model_fe.fc.parameters(), lr=0.001)

model_fe = train_model(model_fe, criterion, optimizer_fe, device, num_epochs=25)